In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 24.5 MB/s eta 0:00:00


In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
!pip install selenium beautifulsoup4 pandas webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_lotteon(search_keyword):
    # Chrome 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 헤드리스 모드
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # 웹드라이버 초기화
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)

    try:
        # URL 설정 및 페이지 로드
        url = f"https://www.lotteon.com/search/search/search.ecn?render=search&platform=pc&q={search_keyword}&mallId"
        driver.get(url)

        # 페이지 로딩 대기
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "s-goods-grid__inside")))
        time.sleep(2)  # 추가 대기시간

        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 상품 리스트 찾기
        products = []
        items = soup.select('ul.s-goods-grid__inside li.s-goods-grid__item')

        for item in items:
            product = {}

            try:
                # 상품 ID와 URL
                product_anchor = item.select_one('a.s-goods__anchor')
                if product_anchor:
                    product['id'] = product_anchor.get('id', '').replace('product-head-', '')
                    product['url'] = product_anchor.get('href')

                # 브랜드 및 상품명
                title_elem = item.select_one('.s-goods-title')
                if title_elem:
                    brand = title_elem.select_one('.s-goods-title__brand')
                    product['brand'] = brand.text.strip() if brand else ''
                    product['title'] = title_elem.get_text(strip=True)

                # 가격 정보
                price_div = item.select_one('.s-goods-price')
                if price_div:
                    # 정상가
                    original = price_div.select_one('.s-goods-price__original .s-goods-price__number')
                    product['original_price'] = original.text.strip() if original else ''

                    # 할인율
                    discount = price_div.select_one('.s-goods-price__discount .s-goods-price__number')
                    product['discount_rate'] = discount.text.strip() if discount else ''

                    # 판매가
                    final = price_div.select_one('.s-goods-price__final .s-goods-price__number')
                    product['final_price'] = final.text.strip() if final else ''

                # 카드 할인 정보
                card_discount = item.select_one('.s-goods-discount')
                product['card_discount'] = card_discount.text.strip() if card_discount else ''

                # 배송 정보
                shipping = item.select_one('.s-goods-additional__item')
                product['shipping'] = shipping.text.strip() if shipping else ''

                products.append(product)

            except Exception as e:
                print(f"상품 정보 추출 중 오류: {e}")
                continue

        # DataFrame 생성 및 CSV 저장
        if products:
            df = pd.DataFrame(products)
            filename = f'lotteon_{search_keyword}_products.csv'
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"데이터가 {filename}에 저장되었습니다.")

        return products

    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")
        return None

    finally:
        driver.quit()

# 실행
if __name__ == "__main__":
    # 엑셀 파일에서 검색어 읽기
    file_name = "keyword2.xlsx"
    try:
        df = pd.read_excel(file_name)
        search_keywords = df['keyword'].dropna().tolist()  # A열에서 검색어 가져오기
    except Exception as e:
        print(f"엑셀 파일을 읽는 중 오류 발생: {e}")
        search_keywords = []

    # 검색어 순차적으로 크롤링
    for search_keyword in search_keywords:
        print(f"\n'{search_keyword}'에 대한 크롤링 시작...")
        results = scrape_lotteon(search_keyword)

        if results:
            print(f"\n총 {len(results)}개의 상품이 검색되었습니다.")


'OB114II'에 대한 크롤링 시작...
데이터가 lotteon_OB114II_products.csv에 저장되었습니다.

총 17개의 상품이 검색되었습니다.

'9B00006M1131004'에 대한 크롤링 시작...
데이터가 lotteon_9B00006M1131004_products.csv에 저장되었습니다.

총 3개의 상품이 검색되었습니다.


In [ ]:
# prompt: 위 코드로 크롤링한 값을 하나의 데이터 프레임형태로 저장

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
!pip install selenium
!pip install requests beautifulsoup4 pandas
!pip install selenium beautifulsoup4 pandas webdriver_manager

def scrape_lotteon(search_keyword):
    # Chrome 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 헤드리스 모드
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # 웹드라이버 초기화
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)

    try:
        # URL 설정 및 페이지 로드
        url = f"https://www.lotteon.com/search/search/search.ecn?render=search&platform=pc&q={search_keyword}&mallId"
        driver.get(url)

        # 페이지 로딩 대기
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "s-goods-grid__inside")))
        time.sleep(2)  # 추가 대기시간

        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 상품 리스트 찾기
        products = []
        items = soup.select('ul.s-goods-grid__inside li.s-goods-grid__item')

        for item in items:
            product = {}

            try:
                # 상품 ID와 URL
                product_anchor = item.select_one('a.s-goods__anchor')
                if product_anchor:
                    product['id'] = product_anchor.get('id', '').replace('product-head-', '')
                    product['url'] = product_anchor.get('href')

                # 브랜드 및 상품명
                title_elem = item.select_one('.s-goods-title')
                if title_elem:
                    brand = title_elem.select_one('.s-goods-title__brand')
                    product['brand'] = brand.text.strip() if brand else ''
                    product['title'] = title_elem.get_text(strip=True)

                # 가격 정보
                price_div = item.select_one('.s-goods-price')
                if price_div:
                    # 정상가
                    original = price_div.select_one('.s-goods-price__original .s-goods-price__number')
                    product['original_price'] = original.text.strip() if original else ''

                    # 할인율
                    discount = price_div.select_one('.s-goods-price__discount .s-goods-price__number')
                    product['discount_rate'] = discount.text.strip() if discount else ''

                    # 판매가
                    final = price_div.select_one('.s-goods-price__final .s-goods-price__number')
                    product['final_price'] = final.text.strip() if final else ''

                # 카드 할인 정보
                card_discount = item.select_one('.s-goods-discount')
                product['card_discount'] = card_discount.text.strip() if card_discount else ''

                # 배송 정보
                shipping = item.select_one('.s-goods-additional__item')
                product['shipping'] = shipping.text.strip() if shipping else ''

                products.append(product)

            except Exception as e:
                print(f"상품 정보 추출 중 오류: {e}")
                continue

        return products

    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")
        return None

    finally:
        driver.quit()

# 실행
if __name__ == "__main__":
    # 엑셀 파일에서 검색어 읽기
    file_name = "keyword2.xlsx"
    try:
        df = pd.read_excel(file_name)
        search_keywords = df['keyword'].dropna().tolist()  # A열에서 검색어 가져오기
    except Exception as e:
        print(f"엑셀 파일을 읽는 중 오류 발생: {e}")
        search_keywords = []

    all_products = [] # 모든 상품 정보를 저장할 리스트

    # 검색어 순차적으로 크롤링
    for search_keyword in search_keywords:
        print(f"\n'{search_keyword}'에 대한 크롤링 시작...")
        results = scrape_lotteon(search_keyword)

        if results:
            all_products.extend(results) # 각 검색어에 대한 결과를 all_products에 추가
            print(f"\n'{search_keyword}' 검색 결과 {len(results)}개 상품 정보 추출 완료")


    # 모든 상품 정보를 하나의 DataFrame으로 만들고 CSV 파일로 저장
    if all_products:
        all_products_df = pd.DataFrame(all_products)
        all_products_df.to_csv('lotteon_all_products.csv', index=False, encoding='utf-8-sig')
        print(f"\n총 {len(all_products)}개의 상품 정보를 lotteon_all_products.csv 파일에 저장했습니다.")
    else:
        print("\n크롤링된 상품 정보가 없습니다.")


'OB114II'에 대한 크롤링 시작...

'OB114II' 검색 결과 17개 상품 정보 추출 완료

'9B00006M1131004'에 대한 크롤링 시작...

'9B00006M1131004' 검색 결과 3개 상품 정보 추출 완료

총 20개의 상품 정보를 lotteon_all_products.csv 파일에 저장했습니다.


In [ ]:
# prompt: lotteon_all_products.csv 를 df로 보기

import pandas as pd

df = pd.read_csv('lotteon_all_products.csv')
df

,id,url,brand,title,original_price,discount_rate,final_price,card_discount,shipping
0,LO2352185301,https://www.lotteon.com/p/product/LO2352185301...,오즈웨어,오즈웨어어그 호주 오즈웨어 코케트 플랫폼 키높이 밴딩 슬리퍼 OB114II PEB,"107,500",10.0,"96,750",삼성카드 7% 할인,무료배송
1,LO2390342814,https://www.lotteon.com/p/product/LO2390342814...,어그,어그호주 어그슬리퍼 겨울 털 코케트 양털 사무실 방한 SLIPPER 14컬러 OB1...,"70,350",9.0,"63,670",하나카드 3% 할인,무료배송
2,LO1978735801,https://www.lotteon.com/p/product/LO1978735801...,오즈웨어,오즈웨어호주 오즈웨어 UGG 우먼 코켓 코케트 방수 양털 어그슬리퍼 OB114II ...,NaN,NaN,"62,900",NaN,무료배송
3,LO2404431196,https://www.lotteon.com/p/product/LO2404431196...,NaN,오즈웨어 호주 방수 코켓 양털 어그슬리퍼 9컬러 OB114II,"86,900",8.0,"79,080",NaN,무료배송
4,LO2370220056,https://www.lotteon.com/p/product/LO2370220056...,오즈웨어,오즈웨어호주 오즈웨어 여성 코케트 양모 양털 플랫폼 천송이 슬리퍼 13컬러 OB114II,NaN,NaN,"72,900",NaN,무료배송
5,LO2363106646,https://www.lotteon.com/p/product/LO2363106646...,오즈웨어,오즈웨어호주어그 24FW 오즈웨어 코케트 슬리퍼 레오파드 프린트 OB114 II L 샌드,"79,000",10.0,"71,100",삼성카드 7% 할인,무료배송
6,LO2363106553,https://www.lotteon.com/p/product/LO2363106553...,오즈웨어,오즈웨어호주어그 24FW 오즈웨어 코케트 슬리퍼 레오파드 프린트 OB114 II L 블랙,"79,000",10.0,"71,100",삼성카드 7% 할인,무료배송
7,LO2363106518,https://www.lotteon.com/p/product/LO2363106518...,오즈웨어,오즈웨어호주어그 24FW 오즈웨어 코케트 슬리퍼 레오파드 프린트 OB114 II L,"79,000",10.0,"71,100",삼성카드 7% 할인,무료배송
8,LO2363106731,https://www.lotteon.com/p/product/LO2363106731...,오즈웨어,오즈웨어호주어그 24FW 오즈웨어 코케트 슬리퍼 레오파드 프린트 OB114 II L...,"79,000",10.0,"71,100",삼성카드 7% 할인,무료배송
9,LO2363106590,https://www.lotteon.com/p/product/LO2363106590...,오즈웨어,오즈웨어호주어그 24FW 오즈웨어 코케트 슬리퍼 레오파드 프린트 OB114 II L...,"79,000",10.0,"71,100",삼성카드 7% 할인,무료배송
